論文  
https://arxiv.org/abs/2212.06013<br>
<br>
GitHub<br>
https://github.com/ml-research/semantic-image-editing<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/SemanticImageEditing_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

## ライブラリのインストール

In [ ]:
%cd /content

!pip install git+https://github.com/ml-research/semantic-image-editing@79d32e1ae8672edca64dca3da0b03b141245bdba
!pip install ftfy

## ライブラリのインポート

In [ ]:
from semdiffusers import SemanticEditPipeline
import torch

device = 'cuda' if torch.cuda.is_available() else "cpu"
print("using device is", device)

access_tokens="ここにAccessTokensを入力"

# 学習済みモデルのセットアップ

In [ ]:
model_id = 'runwayml/stable-diffusion-v1-5'

pipe = SemanticEditPipeline.from_pretrained(
    model_id, use_auth_token=access_tokens
).to(device)

gen = torch.Generator(device=device)

# Text to Image

## prompt設定
prompt参考<br>
https://lexica.art/

In [ ]:
prompt = "photo of cherry blossom tree along a river, cinematic, beautiful, high detail, instagram," #@param {type:"string"}

In [ ]:
SEED = 12
gen.manual_seed(SEED)

# 画像生成
out = pipe(
    prompt = prompt, 
    generator = gen, 
    num_images_per_prompt = 1,
    guidance_scale = 7,
    width = 512,
    height = 512
    )

out.images[0]

In [ ]:
editing_prompts = ['swan on a river']

In [ ]:
gen.manual_seed(SEED)

out = pipe(
    prompt = prompt,
    generator = gen, 
    num_images_per_prompt = 1,
    guidance_scale = 7, 
    width = 512,
    height = 512,
    editing_prompt = editing_prompts,
    reverse_editing_direction=[False, False, False],   # Direction of guidance
    edit_warmup_steps=[20, 10, 11],                    # Warmup period for each concept
    edit_guidance_scale=[2000, 2000, 2000],            # Guidance scale for each concept
    edit_threshold=[-0.2, -0.1, -0.1],                 # Threshold for each concept. Note that positive guidance needs negative thresholds and vice versa
    edit_weights=[1.2,1,1],                            # Weights of the individual concepts against each other
    edit_momentum_scale=0.25,                          # Momentum scale that will be added to the latent guidance
    edit_mom_beta=0.6,                                 # Momentum beta
    )

out.images[0]